In [ ]:
!unzip "/content/millet class.v1i.multiclass.zip" -d "/content/millet_dataset_inspect"


Archive:  /content/millet class.v1i.multiclass.zip
 extracting: /content/millet_dataset_inspect/README.dataset.txt  
 extracting: /content/millet_dataset_inspect/README.roboflow.txt  
   creating: /content/millet_dataset_inspect/test/
 extracting: /content/millet_dataset_inspect/test/B04_jpg.rf.42ee892d41ef68a55d28cd28b253b6e4.jpg  
 extracting: /content/millet_dataset_inspect/test/B100_jpg.rf.6d3c724d92727c5684ebdd5ece9551de.jpg  
 extracting: /content/millet_dataset_inspect/test/B12_jpg.rf.7fbb246bed7d84f1dd99ff0af5294174.jpg  
 extracting: /content/millet_dataset_inspect/test/B18_jpg.rf.ac93d4d6da6edc1ecc59f666031cd688.jpg  
 extracting: /content/millet_dataset_inspect/test/B29_jpg.rf.4a3186e9961070d2fe834620ab7a5b35.jpg  
 extracting: /content/millet_dataset_inspect/test/B34_jpg.rf.112617ab28622adb87ed50fc0a24ace3.jpg  
 extracting: /content/millet_dataset_inspect/test/B43_jpg.rf.b05a997c950da4f821bed24bd36a0dea.jpg  
 extracting: /content/millet_dataset_inspect/test/B46_jpg.rf.903

In [ ]:
import os, shutil

base_dir = "/content/millet_dataset_inspect"

def organize_dataset(split_dir):
    for filename in os.listdir(split_dir):
        if filename.endswith(".jpg"):
            class_name = filename[0]  # first letter of filename
            class_dir = os.path.join(split_dir, class_name)
            os.makedirs(class_dir, exist_ok=True)
            shutil.move(os.path.join(split_dir, filename), os.path.join(class_dir, filename))

for folder in ['train', 'valid', 'test']:
    print(f"Organizing {folder}...")
    organize_dataset(os.path.join(base_dir, folder))

print("✅ Dataset reorganized successfully!")

# check new structure
for root, dirs, files in os.walk(base_dir):
    print(f"{root} -> {len(files)} files")


Organizing train...
Organizing valid...
Organizing test...
✅ Dataset reorganized successfully!
/content/millet_dataset_inspect -> 2 files
/content/millet_dataset_inspect/test -> 1 files
/content/millet_dataset_inspect/test/B -> 17 files
/content/millet_dataset_inspect/test/S -> 11 files
/content/millet_dataset_inspect/test/P -> 5 files
/content/millet_dataset_inspect/test/L -> 8 files
/content/millet_dataset_inspect/test/R -> 8 files
/content/millet_dataset_inspect/train -> 1 files
/content/millet_dataset_inspect/train/B -> 192 files
/content/millet_dataset_inspect/train/S -> 219 files
/content/millet_dataset_inspect/train/P -> 198 files
/content/millet_dataset_inspect/train/L -> 213 files
/content/millet_dataset_inspect/train/R -> 219 files
/content/millet_dataset_inspect/valid -> 1 files
/content/millet_dataset_inspect/valid/B -> 19 files
/content/millet_dataset_inspect/valid/S -> 15 files
/content/millet_dataset_inspect/valid/P -> 28 files
/content/millet_dataset_inspect/valid/L -> 

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_data = datasets.ImageFolder(root=os.path.join(base_dir, 'train'), transform=transform)
val_data = datasets.ImageFolder(root=os.path.join(base_dir, 'valid'), transform=transform)
test_data = datasets.ImageFolder(root=os.path.join(base_dir, 'test'), transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)
print("Number of training images:", len(train_data))
print("Number of validation images:", len(val_data))
print("Number of test images:", len(test_data))


Classes: ['B', 'L', 'P', 'R', 'S']
Number of training images: 1041
Number of validation images: 99
Number of test images: 49


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

base_dir = "/content/millet_dataset_inspect"

# Augmentations for training (to improve generalization)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Validation and Test: no augmentation, just resize + normalize
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_data = datasets.ImageFolder(root=os.path.join(base_dir, 'train'), transform=train_transform)
val_data = datasets.ImageFolder(root=os.path.join(base_dir, 'valid'), transform=val_test_transform)
test_data = datasets.ImageFolder(root=os.path.join(base_dir, 'test'), transform=val_test_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

print("Classes:", train_data.classes)
print("Training samples:", len(train_data))
print("Validation samples:", len(val_data))
print("Testing samples:", len(test_data))


Classes: ['B', 'L', 'P', 'R', 'S']
Training samples: 1041
Validation samples: 99
Testing samples: 49


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.vit_b_16(weights="IMAGENET1K_V1")  # Pretrained Vision Transformer
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers initially

# Replace the final classification head
num_classes = len(train_data.classes)
model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)

model = model.to(device)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:02<00:00, 146MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.heads.head.parameters(), lr=0.0005)


In [ ]:
epochs = 10  # you can adjust later

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    val_acc = 0

    # Validation
    model.eval()
    with torch.no_grad():
        correct_val = 0
        total_val = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
        val_acc = 100 * correct_val / total_val

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")


Epoch 1/10: 100%|██████████| 33/33 [13:57<00:00, 25.39s/it]


Epoch [1/10] - Loss: 1.4457, Train Acc: 40.92%, Val Acc: 51.52%


Epoch 2/10: 100%|██████████| 33/33 [13:53<00:00, 25.27s/it]


Epoch [2/10] - Loss: 1.1069, Train Acc: 66.57%, Val Acc: 56.57%


Epoch 3/10: 100%|██████████| 33/33 [13:51<00:00, 25.18s/it]


...

Epoch [8/10] - Loss: 0.5769, Train Acc: 84.63%, Val Acc: 71.72%


Epoch 9/10: 100%|██████████| 33/33 [13:43<00:00, 24.95s/it]


Epoch [9/10] - Loss: 0.5356, Train Acc: 87.32%, Val Acc: 74.75%


Epoch 10/10: 100%|██████████| 33/33 [13:43<00:00, 24.95s/it]


Epoch [10/10] - Loss: 0.5096, Train Acc: 86.46%, Val Acc: 76.77%


In [ ]:
torch.save(model.state_dict(), "millet_vit_model.pth")
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [ ]:
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_acc = 100 * correct_test / total_test
print(f"🎯 Test Accuracy: {test_acc:.2f}%")


🎯 Test Accuracy: 73.47%


In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/millet_vit_model.pth")
print("✅ Model saved to Google Drive!")


✅ Model saved to Google Drive!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model.load_state_dict(torch.load("/content/drive/MyDrive/millet_vit_model.pth", map_location=device))
model.eval()
print("✅ Model loaded successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model loaded successfully!


To run the model just run the below code


In [1]:
from google.colab import drive
drive.mount('/content/drive')

import torch
from torchvision import models

# (Recreate your model structure same as before)
model = models.vit_b_16(weights=None)
num_features = model.heads.head.in_features
model.heads.head = torch.nn.Linear(num_features, 5)  # 5 classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("/content/drive/MyDrive/millet_vit_model.pth", map_location=device))
model.to(device)
model.eval()

print("✅ Model loaded successfully!")


Mounted at /content/drive
✅ Model loaded successfully!


In [2]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.2 MB/s eta 0:00:00


In [3]:
!cp /content/drive/MyDrive/millet_vit_model.pth .


In [6]:
# %%writefile app.py
# import streamlit as st
# import torch
# from torchvision import transforms
# from PIL import Image
# import torch.nn as nn
# import torchvision.models as models

# # Device configuration
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load model
# model = models.vit_b_16(weights=None)
# num_classes = 5  # B, L, P, R, S
# model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)
# model.load_state_dict(torch.load("/content/drive/MyDrive/millet_vit_model.pth", map_location=device))
# model.eval()

# # Define transform
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

# # Class names
# class_names = ['Buck Wheat Millet', 'Little Millet', 'Pearl Millet', 'Ragi MIllet', 'Sorghum Millet']

# # Streamlit UI
# st.title("🌾 Millet Classification App")
# st.write("Upload an image of millet to identify its type.")

# uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

# if uploaded_file:
#     image = Image.open(uploaded_file).convert('RGB')
#     st.image(image, caption='Uploaded Image', use_container_width=True)

#     # Preprocess
#     img_tensor = transform(image).unsqueeze(0).to(device)

#     # Predict
#     with torch.no_grad():
#         outputs = model(img_tensor)
#         _, predicted = torch.max(outputs, 1)
#         confidence = torch.nn.functional.softmax(outputs, dim=1)[0] * 100

#     st.write(f"### 🧠 Predicted Class: **{class_names[predicted.item()]}**")
#     st.write("#### Confidence Levels:")
#     for i, c in enumerate(class_names):
#         st.write(f"- {c}: {confidence[i]:.2f}%")


Overwriting app.py


In [9]:
%%writefile app.py
import streamlit as st
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model
model = models.vit_b_16(weights=None)
num_classes = 5  # B, L, P, R, S
model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/millet_vit_model.pth", map_location=device))
model.eval()

# Define transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Class names
class_names = ['Buck Wheat Millet', 'Little Millet', 'Pearl Millet', 'Ragi MIllet', 'Sorghum Millet']

# Streamlit UI
st.title("🌾 Millet Classification App")
st.write("Upload an image of millet to identify its type.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption='Uploaded Image', use_container_width=True)

    # Preprocess
    img_tensor = transform(image).unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
        confidence = torch.nn.functional.softmax(outputs, dim=1)[0] * 100

    st.write(f"### 🧠 Predicted Class: **{class_names[predicted.item()]}**")
    st.write("#### Confidence Levels:")
    for i, c in enumerate(class_names):
        st.write(f"- {c}: {confidence[i]:.2f}%")

    # 🌾 Detailed Millet Information
    millet_info = {
        "Buck Wheat Millet": """
        **Buck Wheat Millet**
        - Gluten-free and rich in high-quality protein and fiber.
        - Helps lower cholesterol and improves heart health.
        - Contains essential amino acids and antioxidants.
        - Beneficial for diabetics due to its low glycemic index.
        - Commonly used in pancakes, noodles, and porridge.
        """,

        "Little Millet": """
        **Little Millet**
        - Packed with iron, calcium, and B vitamins.
        - Supports immunity and boosts metabolism.
        - Helps in managing cholesterol and diabetes.
        - Provides sustained energy and promotes digestion.
        - Used in pulao, pongal, and breakfast dishes.
        """,

        "Pearl Millet": """
        **Pearl Millet (Bajra)**
        - Excellent source of iron and protein.
        - Strengthens bones and boosts energy.
        - Good for heart health and controls blood sugar.
        - Rich in antioxidants that protect against diseases.
        - Commonly used in rotis and porridge.
        """,

        "Ragi MIllet": """
        **Ragi Millet (Finger Millet)**
        - Rich in calcium, iron, and dietary fiber.
        - Strengthens bones and aids in weight management.
        - Great for diabetics due to low glycemic index.
        - Boosts hemoglobin levels and improves digestion.
        - Used in rotis, malt drinks, and porridge.
        """,

        "Sorghum Millet": """
        **Sorghum Millet (Jowar)**
        - Gluten-free grain high in fiber and protein.
        - Improves digestion and heart health.
        - Helps regulate blood sugar levels.
        - Contains antioxidants and vitamins for immunity.
        - Used in rotis, dosas, and baked snacks.
        """
    }

    # Display millet information
    predicted_millet = class_names[predicted.item()]
    st.markdown("### 🌿 Millet Information")
    st.markdown(millet_info.get(predicted_millet, "Information not available."))


Overwriting app.py


In [10]:
# # Install required packages
# !pip install streamlit pyngrok -q

# import os
# import time
# import threading
# from pyngrok import ngrok

# # ✅ Replace with your Ngrok token
# NGROK_AUTH_TOKEN = "34zuwkBIU4evUk6kYHbR2PJk6bu_3UTPorRDHinPVeNTvHWKN"

# # ✅ Create Streamlit app file
# with open("app.py", "w") as f:
#     f.write("""
# import streamlit as st
# import torch
# from torchvision import transforms
# from PIL import Image
# import torch.nn as nn
# from torchvision import models

# st.title("🌾 Millet Classification App")
# st.write("Upload an image to classify its millet type!")

# # Load your trained model
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = models.vit_b_16(weights=None)
# model.heads.head = nn.Linear(model.heads.head.in_features, 5)
# model.load_state_dict(torch.load('/content/drive/MyDrive/millet_vit_model.pth', map_location=device))
# model.to(device) # Add this line to move the model to the correct device
# model.eval()

# # Define transform
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

# classes = ['Buck Wheat Millet', 'Little Millet', 'Pearl Millet', 'Ragi MIllet', 'Sorghum Millet']

# uploaded_file = st.file_uploader("Choose a millet image...", type=["jpg", "jpeg", "png"])
# if uploaded_file is not None:
#     image = Image.open(uploaded_file).convert('RGB')
#     st.image(image, caption="Uploaded Image", use_column_width=True)

#     img_tensor = transform(image).unsqueeze(0).to(device)

#     with torch.no_grad():
#         outputs = model(img_tensor)
#         _, predicted = torch.max(outputs, 1)
#         st.success(f"Predicted Millet Type: {classes[predicted.item()]}")
# """)

# # ✅ Set Ngrok token and expose Streamlit port
# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Kill any running ngrok processes to avoid conflicts
# ngrok.kill()

# public_url = ngrok.connect(8501)
# print("🌐 Streamlit App URL:", public_url.public_url)

# # ✅ Launch Streamlit app in a separate thread
# def run_app():
#     os.system("streamlit run app.py")

# thread = threading.Thread(target=run_app)
# thread.start()

# # Optional: wait for app to start
# time.sleep(10)

🌐 Streamlit App URL: https://nonsynthetically-phylacterical-zachariah.ngrok-free.dev


In [12]:
# Install required packages
!pip install streamlit pyngrok -q

import os
import time
import threading
from pyngrok import ngrok

# ✅ Replace with your Ngrok token
NGROK_AUTH_TOKEN = "34zuwkBIU4evUk6kYHbR2PJk6bu_3UTPorRDHinPVeNTvHWKN"

# ✅ Create Streamlit app file
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
from torchvision import models

# 🌾 Streamlit UI
st.title("🌾 Millet Classification App")
st.write("Upload an image to classify its millet type and learn more about it!")

# ✅ Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.vit_b_16(weights=None)
model.heads.head = nn.Linear(model.heads.head.in_features, 5)
model.load_state_dict(torch.load('/content/drive/MyDrive/millet_vit_model.pth', map_location=device))
model.to(device)
model.eval()

# ✅ Define transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# ✅ Millet class names
classes = ['Buck Wheat Millet', 'Little Millet', 'Pearl Millet', 'Ragi Millet', 'Sorghum Millet']

# ✅ Millet descriptions
millet_info = {
    'Buck Wheat Millet': (
        '''🍞 **Buckwheat Millet**
- A gluten-free seed rich in antioxidants and protein.
- Great for heart health and maintaining cholesterol levels.
- Helps control blood sugar due to its low glycemic index.
- Commonly used in pancakes, noodles, and porridge.
- Boosts immunity and aids digestion.'''
    ),
    'Little Millet': (
        '''🌾 **Little Millet**
- Rich in iron, dietary fiber, and minerals.
- Ideal for diabetics as it helps regulate blood sugar levels.
- Supports heart health and aids in weight management.
- Can be used in upma, pongal, and khichdi.
- Keeps you full for longer and promotes good digestion.'''
    ),
    'Pearl Millet': (
        '''🌻 **Pearl Millet (Bajra)**
- Packed with protein, calcium, and iron.
- Helps reduce cholesterol and control diabetes.
- Known for its high energy and heat-generating properties.
- Commonly used in making rotis and porridge.
- Strengthens bones and improves digestion.'''
    ),
    'Ragi Millet': (
        '''🍪 **Finger Millet (Ragi)**
- A rich source of calcium, iron, and amino acids.
- Great for bone health and growth in children.
- Helps in controlling diabetes and obesity.
- Commonly used to make ragi malt, roti, and dosa.
- Keeps you energized and supports skin health.'''
    ),
    'Sorghum Millet': (
        '''🌿 **Sorghum Millet (Jowar)**
- High in fiber, protein, and essential nutrients.
- Improves digestion and boosts metabolism.
- Naturally gluten-free and good for celiac patients.
- Helps manage blood sugar and promotes heart health.
- Commonly used in bhakri, porridge, and snacks.'''
    )
}


# ✅ File uploader
uploaded_file = st.file_uploader("Choose a millet image...", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption="Uploaded Image", use_column_width=True)

    img_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = classes[predicted.item()]

    # ✅ Show results
    st.success(f"✅ Predicted Millet Type: **{predicted_class}**")
    st.markdown("---")
    st.markdown(millet_info[predicted_class])
""")

# ✅ Set Ngrok token and expose Streamlit port
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()  # kill previous tunnels

public_url = ngrok.connect(8501)
print("🌐 Streamlit App URL:", public_url.public_url)

# ✅ Launch Streamlit app in a separate thread
def run_app():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run_app)
thread.start()

time.sleep(10)


🌐 Streamlit App URL: https://nonsynthetically-phylacterical-zachariah.ngrok-free.dev
